In [1]:
import pandas as pd
import import_ipynb
from common import *

importing Jupyter notebook from common.ipynb


|종목 수|시가총액 하위(%)|기간|리밸런싱|CAGRR|IFRS|CFS|NOTE|
|-----|-------|-------|-----|-----|----|---|-|
| 30 | 20 | 2002 ~ 2019 |8/1| 1.426987 |1|0, 1| |
| 30 | 20 | 2002 ~ 2019 |10/1| 1.462075 |1|0, 1| |
| 30 | 20 | 2002 ~ 2019 |12/1| 1.424886 |1|0, 1| |

In [7]:
NUM_STOCKS = 30
MIN_MARKET_CAP = 0.0
MAX_MARKET_CAP = 0.2

In [8]:
def get_stocks(filepath, date):
    print("------------------------------")
    print(date, filepath)
    df = pd.read_csv(filepath, dtype={"기업코드":"string", "종목코드":"string"})
    
#     past_income_cols = []
#     last_year = int(date[:4]) - 1
#     for i in range(1, 6):
#         year = last_year - i
#         col_name = str(year) + '-' + '당기순이익'
#         past_income_cols.append(col_name)
        
    cols = ['종목코드', 'IFRS', 'CFS', '회사명', '시가총액', 'PBR', 'GP/A', '당기순이익']
    #cols.extend(past_income_cols)

    df_qp = df[cols]
    print('전체', len(df_qp))
    
    # 지주사, 금융사 제외
    df_qp = exclude_holdings_and_finances(df_qp, '회사명')
    print('지주사, 금융사 제외', len(df_qp))
    
    # 국외주식 제외
    df_qp = exclude_foreign_corps(df_qp, '종목코드')
    print('국외주식 제외', len(df_qp))
    
    # 당기순이익 최근 5년 +인것만
    #df_qp = exclude_minus_income_corps(df_qp, past_income_cols)
    
    # 전년도 당기순이익 +인것만
    # df_qp = df_qp[df_qp['당기순이익'] > 0]
    
    # 시가총액 하위 20% 
    #df_qp = df_qp[(df_qp['IFRS'] == 1) & (df_qp['CFS'] == 1)] # 국제회계기준 + 연결재무제표 기업만
    df_qp = df_qp[df_qp['시가총액'] > 0] # 시가총액 data가 없는 row는 제거
    df_qp = df_qp.sort_values(by=['시가총액'])
    print('가격정보 없는 기업 제외', len(df_qp))
    df_qp = df_qp[int(len(df_qp)*MIN_MARKET_CAP):int(len(df_qp)*MAX_MARKET_CAP)]
    
    df_qp = df_qp[df_qp['PBR'] > 0]
    
    # get ranks
    df_qp['1/PBR'] = 1 / df_qp['PBR']
    df_qp['RANK_1'] = df_qp['1/PBR'].rank(ascending=False)
    df_qp['RANK_2'] = df_qp['GP/A'].rank(ascending=False)
    
    df_qp['TOTAL_RANK'] = df_qp['RANK_1'] + df_qp['RANK_2']

    df_qp = df_qp.sort_values(by=['TOTAL_RANK'])    
    
    stocks = []
    counter = 0
    for i, row in df_qp.iterrows():
        counter += 1
        if counter > NUM_STOCKS:
            break
        
        candidate = row['종목코드']
        candidate_name = row['회사명']
            
        #print(candidate_name, row['TOTAL_RANK'])
        stocks.append(candidate)
    
    print("선정 기업 수", len(stocks))
    return stocks

In [10]:
# get_stocks('data/daily/8-1/2018-08-01.csv', '2018-08-01')